In [134]:
import numpy as np
import pandas as pd

In [135]:
from pyecharts import Kline, Overlap, Line, Grid

In [136]:
class Point:
    def __init__(self, time_index, value, drt):
        self.TmIdx, self.V, self.drt = time_index, value, drt
        
    def print_(self):
        print('P[',self.TmIdx, self.V, self.drt, end='] ')

In [137]:
class StdK:
    def __init__(self, **kwargs):
        self.drt = kwargs['drt']
        if 'k_bar' in kwargs.keys():
            self.H, self.L, self.TmIdx = kwargs['k_bar'][1], kwargs['k_bar'][2], kwargs['k_bar'][4]
        if 'H' in kwargs.keys():
            self.H, self.L, self.TmIdx = kwargs['H'], kwargs['L'], kwargs['TmIdx']
        self.merged = 1
        # self.range = None  # for ES, trend_drt==1 ~ rangeL, trend_drt==-1 ~ rangeH
    
    def update(self, k_bar):
        flag = 0 # 0:stain; 1: up new; -1: down new
        if (k_bar[1] > self.H and k_bar[2] > self.L) or (self.drt == 1 and k_bar[1] > self.H):
            flag = 1
            k_bar[2] = max(k_bar[2], self.L)
        elif (k_bar[1] < self.H and k_bar[2] < self.L) or (self.drt == -1 and k_bar[2] < self.L):
            flag = -1
            k_bar[1] = min(k_bar[1], self.H)
        else:
            flag = 0
            if self.drt == -1:
                self.H = min(self.H, k_bar[1])
            elif self.drt == 1:
                self.L = max(self.L, k_bar[2])
        return flag, k_bar 
    
    def trimUpdate(self, new_stdk):
        if self.drt == 1:
            self.L = max(self.L, new_stdk['L'])
        elif self.drt == -1:
            self.H = min(self.H, new_stdk['H'])
        else:
            self.L = max(self.L, new_stdk['L'])
            self.H = min(self.H, new_stdk['H'])
    
    def print_(self):
        print('Std[',self.TmIdx, self.H, self.L, self.drt, end='] ')

                

In [97]:
std_L = [StdK(drt=0, k_bar=dt[0])]
for k in dt[1:]:
    new, new_std_k = std_L[-1].update(k)
    if new != 0:
        std_L.append(StdK(k_bar=new_std_k, drt=new))
        std_L[-1].print_()
        
len(std_L)

Std[ 1.0 5.09 5.04 1] Std[ 4.0 5.07 5.05 -1] Std[ 6.0 5.07 5.04 -1] Std[ 8.0 5.07 5.03 -1] Std[ 10.0 5.04 5.01 -1] Std[ 11.0 5.03 5.0 -1] Std[ 14.0 5.01 4.96 -1] Std[ 16.0 4.96 4.93 -1] Std[ 17.0 4.96 4.9 -1] Std[ 18.0 4.92 4.87 -1] Std[ 19.0 4.89 4.83 -1] Std[ 22.0 4.86 4.81 -1] Std[ 23.0 4.83 4.74 -1] Std[ 24.0 4.8 4.73 -1] Std[ 27.0 4.75 4.72 -1] Std[ 28.0 4.75 4.69 -1] Std[ 29.0 4.7 4.61 -1] Std[ 31.0 4.75 4.68 1] Std[ 32.0 4.78 4.72 1] Std[ 38.0 4.76 4.72 -1] Std[ 39.0 4.84 4.73 1] Std[ 40.0 4.87 4.8 1] Std[ 41.0 4.84 4.78 -1] Std[ 43.0 4.84 4.8 1] Std[ 46.0 4.87 4.8 1] Std[ 47.0 4.9 4.86 1] Std[ 49.0 4.88 4.85 -1] Std[ 51.0 4.86 4.84 -1] Std[ 54.0 4.86 4.81 -1] Std[ 55.0 4.82 4.8 -1] Std[ 56.0 4.82 4.79 -1] Std[ 58.0 4.81 4.77 -1] Std[ 59.0 4.86 4.78 1] Std[ 61.0 4.81 4.77 -1] Std[ 62.0 4.82 4.79 1] Std[ 63.0 4.85 4.8 1] Std[ 65.0 4.82 4.79 -1] Std[ 66.0 4.81 4.77 -1] Std[ 68.0 4.82 4.8 1] Std[ 70.0 4.81 4.77 -1] Std[ 73.0 4.81 4.78 1] Std[ 77.0 4.84 4.79 1] Std[ 79.0 4.85 4.82 1

275

In [98]:
# draw stdk
bar_L = []
for std in std_L:
    bar = [std.TmIdx]
    if std.drt == 1:
        bar.append(std.L)
        bar.append(std.H)
        bar.append(std.L)
        bar.append(std.H)
    else:
        bar.append(std.H)
        bar.append(std.L)
        bar.append(std.H)
        bar.append(std.L) 
    bar_L.append(bar)
df_tmp = pd.DataFrame(bar_L, columns=['TmIdx', 'O', 'C', 'L', 'H'])
df_tmp['TmIdx2'] = df_tmp['TmIdx'].astype('int32')
df_tmp.drop('TmIdx', axis=1, inplace=True)
df_tmp2 = x.merge(df_tmp, how='left', left_on='TmIdx', right_on='TmIdx2')
df_tmp2.fillna('', inplace=True)
std_dt = df_tmp2[['O', 'C', 'L', 'H']].values.tolist()
        

In [143]:
lv0_L =[Stick('init', k_bar=dt[0])]
for k in dt[1:]:
    print(k[4],':', end='')
    new, new_stick = lv0_L[-1].update(k)
    if new == 2:
        print("!!!!!!!!!!!")
        lv0_L.append(Stick('trim', **new_stick))
    print('')   
        
len(lv0_L)

1.0 :
2.0 :
3.0 :
4.0 :
5.0 :
6.0 :
7.0 :
8.0 :
9.0 :
10.0 :
11.0 :
12.0 :
13.0 :
14.0 :
15.0 :
16.0 :
17.0 :
18.0 :
19.0 :
20.0 :
21.0 :
22.0 :
23.0 :
24.0 :!!!!!!!!!!!

25.0 :
26.0 :!!!!!!!!!!!

27.0 :
28.0 :
29.0 :
30.0 :
31.0 :
32.0 :
33.0 :
34.0 :
35.0 :
36.0 :
37.0 :!!!!!!!!!!!

38.0 :
39.0 :
40.0 :
41.0 :
42.0 :!!!!!!!!!!!

43.0 :
44.0 :
45.0 :
46.0 :
47.0 :
48.0 :
49.0 :
50.0 :
51.0 :!!!!!!!!!!!

52.0 :
53.0 :
54.0 :
55.0 :
56.0 :!!!!!!!!!!!

57.0 :
58.0 :
59.0 :
60.0 :
61.0 :
62.0 :
63.0 :
64.0 :!!!!!!!!!!!

65.0 :
66.0 :
67.0 :
68.0 :
69.0 :
70.0 :
71.0 :
72.0 :!!!!!!!!!!!

73.0 :
74.0 :
75.0 :
76.0 :!!!!!!!!!!!

77.0 :
78.0 :
79.0 :
80.0 :
81.0 :
82.0 :
83.0 :
84.0 :
85.0 :
86.0 :
87.0 :
88.0 :
89.0 :
90.0 :
91.0 :
92.0 :
93.0 :
94.0 :
95.0 :
96.0 :
97.0 :
98.0 :
99.0 :
100.0 :
101.0 :
102.0 :
103.0 :
104.0 :!!!!!!!!!!!

105.0 :
106.0 :
107.0 :
108.0 :
109.0 :
110.0 :
111.0 :
112.0 :
113.0 :
114.0 :
115.0 :!!!!!!!!!!!

116.0 :
117.0 :
118.0 :
119.0 :
120.0 :
121.0 :
122.0 :


1196.0 :
1197.0 :
1198.0 :
1199.0 :
1200.0 :
1201.0 :
1202.0 :
1203.0 :
1204.0 :
1205.0 :
1206.0 :
1207.0 :
1208.0 :
1209.0 :
1210.0 :
1211.0 :!!!!!!!!!!!

1212.0 :
1213.0 :
1214.0 :
1215.0 :
1216.0 :!!!!!!!!!!!

1217.0 :
1218.0 :
1219.0 :
1220.0 :
1221.0 :
1222.0 :
1223.0 :!!!!!!!!!!!

1224.0 :
1225.0 :
1226.0 :
1227.0 :
1228.0 :
1229.0 :
1230.0 :
1231.0 :
1232.0 :
1233.0 :!!!!!!!!!!!

1234.0 :
1235.0 :
1236.0 :
1237.0 :
1238.0 :
1239.0 :
1240.0 :
1241.0 :
1242.0 :
1243.0 :!!!!!!!!!!!

1244.0 :
1245.0 :
1246.0 :
1247.0 :
1248.0 :
1249.0 :
1250.0 :
1251.0 :
1252.0 :!!!!!!!!!!!

1253.0 :
1254.0 :
1255.0 :
1256.0 :
1257.0 :
1258.0 :
1259.0 :!!!!!!!!!!!

1260.0 :
1261.0 :
1262.0 :
1263.0 :
1264.0 :
1265.0 :
1266.0 :!!!!!!!!!!!

1267.0 :
1268.0 :
1269.0 :
1270.0 :
1271.0 :
1272.0 :
1273.0 :
1274.0 :!!!!!!!!!!!

1275.0 :
1276.0 :
1277.0 :
1278.0 :
1279.0 :!!!!!!!!!!!

1280.0 :
1281.0 :
1282.0 :
1283.0 :
1284.0 :
1285.0 :
1286.0 :
1287.0 :
1288.0 :
1289.0 :
1290.0 :
1291.0 :
1292.0 :
1293.0 

121

In [144]:
tm, v, stick_drt = [], [], []
for stick in lv0_L:
    tm.append(stick.start.TmIdx)
    v.append(stick.start.V)
    stick.print_()
    print('')

drt: 1,status:2,rc: 4,S:[  0.0,3708.0, 0], P:[ 17.0,3710.0,-1],pp: 2
drt:-1,status:2,rc: 4,S:[ 17.0,3710.0,-1], P:[ 21.0,3700.0, 1],pp: 2
drt: 1,status:2,rc: 4,S:[ 21.0,3700.0, 1], P:[ 33.0,3710.0,-1],pp: 4
drt:-1,status:2,rc: 8,S:[ 33.0,3710.0,-1], P:[ 35.0,3702.0, 1],pp: 2
drt: 1,status:2,rc: 6,S:[ 35.0,3702.0, 1], P:[ 40.0,3716.0,-1],pp: 4
drt:-1,status:2,rc: 4,S:[ 40.0,3716.0,-1], P:[ 49.0,3706.0, 1],pp: 3
drt: 1,status:2,rc: 4,S:[ 49.0,3706.0, 1], P:[ 54.0,3710.0,-1],pp: 2
drt:-1,status:2,rc: 4,S:[ 54.0,3710.0,-1], P:[ 63.0,3702.0, 1],pp: 2
drt: 1,status:2,rc: 4,S:[ 63.0,3702.0, 1], P:[ 66.0,3708.0,-1],pp: 2
drt:-1,status:2,rc: 6,S:[ 66.0,3708.0,-1], P:[ 94.0,3686.0, 1],pp: 8
drt: 1,status:2,rc: 4,S:[ 94.0,3686.0, 1], P:[108.0,3700.0,-1],pp: 7
drt:-1,status:2,rc: 6,S:[108.0,3700.0,-1], P:[122.0,3690.0, 1],pp: 6
drt: 1,status:2,rc: 6,S:[122.0,3690.0, 1], P:[136.0,3704.0,-1],pp:10
drt:-1,status:2,rc: 4,S:[136.0,3704.0,-1], P:[140.0,3694.0, 1],pp: 3
drt: 1,status:2,rc: 4,S:[140.0,369

In [138]:
class Stick:
    def __init__(self, method, **kwargs):
        if method == 'init':
            #self.TmIdx_start, self.TmIdx_end = kwargs['k_bar'][4], kwargs['k_bar'][4]
            #self.O, self.C = kwargs['k_bar'][0], kwargs['k_bar'][3]
            self.start = Point(kwargs['k_bar'][4], kwargs['k_bar'][0], drt=0)
            self.end = Point(kwargs['k_bar'][4], kwargs['k_bar'][0], drt=0)
            self.range_value = kwargs['k_bar'][3]
            self.std_k_bar_stack = [StdK(k_bar=kwargs['k_bar'], drt=0)]
            self.peak = self.getPoint(0, 'init')
            self.peak_position = 0
            self.drt = 0
            
            self.reverse_count = 0
            self.status = 0 # 0:open; 1:ripe; 2:close
        elif method == 'trim':
            self.std_k_bar_stack = kwargs['std_k_bar_stack']
            self.drt = -self.std_k_bar_stack[0].drt
            self.peak_position = kwargs['peak_position']
            if self.drt == 1:
                self.O, self.C = self.std_k_bar_stack[0].L, self.std_k_bar_stack[-1].H
                self.start = self.getPoint(0, 'L')
                self.end = self.getPoint(-1, 'L')
                self.peak = self.getPoint(self.peak_position, 'H')
            elif self.drt == -1:
                self.O, self.C = self.std_k_bar_stack[0].H, self.std_k_bar_stack[-1].L
                self.start = self.getPoint(0, 'H')
                self.end = self.getPoint(-1, 'H')
                self.peak = self.getPoint(self.peak_position, 'L')
            self.range_value = self.peak.V
            self.reverse_count = 1
            self.status = 1 # 0:open, new peak与起点比较; 1:ripe, new peak与self.peak比较; 2:close
            #print('!!!!!!!!!!!!')
            #print('1st stk bar: [{},{},{},{}]'.format(self.std_k_bar_stack[0].TmIdx, 
            #                                      self.std_k_bar_stack[0].H,self.std_k_bar_stack[0].L,
            #                                         self.std_k_bar_stack[0].drt))
            # self.print_()
    
    def update(self, k_bar):
        self.TmIdx_end, self.C = k_bar[4], k_bar[3]
        flag = 0 # 0:stain; 1: updated; 2:cloes
        new_stick = ''
        if self.updateStdK(k_bar) != 0: # 更新stdK, stdK_stack，如果有新的stdK，则继续更新Stick
            peak, pp = self.findPeak()
            #print('peak:', peak, pp, end='')
            if peak == 0:
                flag = self.updateRange(k_bar)
                self.reverse_count += 1
                
            elif self.status == 0:
                
                if self.drt == 0:
                    self.drt = peak
                elif peak == self.drt:
                    flag = 0
                    # print('peak == self.drt', end='')
                elif self.reverse_count > 3:
                    self.status = 1
                    self.peak_position = pp
                    self.peak = self.getPoint(pp)
                    self.reverse_count = 1
                    flag = 1
                #print('status:0-', self.status, ' peak:', peak, ' drt:', self.drt, end=' ')                
            # Ripe, new peak 与 peak比较
            elif peak == self.peak.drt: # 
                #print('[Peak!=drt]drt:{}, peak:{}, new peak:{}'.format(self.drt, self.peak.drt, peak), end='/')
                if self.drt == 1 and self.std_k_bar_stack[-2].H >= self.peak.V:
                    self.peak_position = pp
                    self.peak = self.getPoint(pp)
                    self.reverse_count = 1
                    flag = 1                    
                elif self.drt == -1 and self.std_k_bar_stack[-2].L <= self.peak.V:
                    self.peak_position = pp
                    self.peak = self.getPoint(pp)
                    self.reverse_count = 1
                    flag = 1 
                else:
                    self.reverse_count += 1
                    flag = 0
            elif peak != self.peak.drt and self.reverse_count > 3:
                self.closeStick()
                new_stick = self.newStick(self.peak_position)
                flag = 2 
                #print('status:{},new peak!=peak.drt:{},{}, old stick drt:{},pp:{},'\
                #      .format(self.status, peak, self.peak.drt, self.drt, self.peak_position), end='/')
            # print('flag=', flag, end='')
        # self.print_()
        return flag, new_stick
    
    def updateStdK(self, k_bar):
        is_new = 0
        is_new, trimed_k_bar = self.std_k_bar_stack[-1].update(k_bar)
        if is_new != 0:
            # print('new std/ ', end='')
            self.std_k_bar_stack.append(StdK(k_bar=trimed_k_bar, drt=is_new)) 
            self.reverse_count += 1
            #self.std_k_bar_stack[-1].print_()
        return is_new
            
    def save(self):
        return None
    
    #=================================================================================
    def findPeak(self):
        peak_type = 0
        if self.reverse_count > 2:
            if (self.std_k_bar_stack[-2].H > self.std_k_bar_stack[-1].H and 
               self.std_k_bar_stack[-2].H > self.std_k_bar_stack[-3].H):
                peak_type = -1
            elif (self.std_k_bar_stack[-2].L < self.std_k_bar_stack[-1].L and 
               self.std_k_bar_stack[-2].L < self.std_k_bar_stack[-3].L):
                peak_type = 1
        return peak_type, len(self.std_k_bar_stack) - 2
    
    def updateRange(self, k_bar):
        flag = 0  # 0: stick stain; 1: stick updated
        if self.peak.drt == 0:
            return flag
        if self.drt == 1 and k_bar[1] >= self.peak.V:
            self.range_value = k_bar[1]
            self.status = 0
            flag = 1
        elif self.drt == -1 and k_bar[2] <= self.peak.V:
            self.range_value = k_bar[2]
            self.status = 0 
            flag = 1
        # if flag == 1:
            # print('RangeUpdated!!', end='')
        return flag
    
    def closeStick(self):
        self.status = 2
        return None
    
    def newStick(self, peak_position_in_std_stack):
        std_k_bar_stack = self.std_k_bar_stack[peak_position_in_std_stack:]
        peak_position = len(std_k_bar_stack) - 2

        new_stick = {
            'std_k_bar_stack' : std_k_bar_stack,
            'peak_position' : peak_position
        }
        return new_stick
    
    def getPoint(self, position_in_std_stack, method=None):
        if method is None:
            drt = self.std_k_bar_stack[position_in_std_stack].drt
            if drt == 1:
                p = Point(self.std_k_bar_stack[position_in_std_stack].TmIdx,
                     self.std_k_bar_stack[position_in_std_stack].H,
                     -1)
            elif drt == -1:
                p = Point(self.std_k_bar_stack[position_in_std_stack].TmIdx,
                     self.std_k_bar_stack[position_in_std_stack].L,
                     1)
        elif method == 'H':
            p = Point(self.std_k_bar_stack[position_in_std_stack].TmIdx,
                     self.std_k_bar_stack[position_in_std_stack].H,
                     -1)
        elif method == 'L':
            p = Point(self.std_k_bar_stack[position_in_std_stack].TmIdx,
                     self.std_k_bar_stack[position_in_std_stack].L,
                     1)
        elif method == 'init':
            p = Point(self.std_k_bar_stack[position_in_std_stack].TmIdx,
                     self.std_k_bar_stack[position_in_std_stack].L,
                     0)
        return p
    
    def beRipe(self):
        self.status == 1
        
    def print_(self):
        print('drt:{:>2},status:{},rc:{:>2},S:[{:>5},{:>6},{:>2}], P:[{:>5},{:>6},{:>2}],pp:{:>2}'\
              .format(self.drt, self.status, self.reverse_count
             ,self.start.TmIdx, self.start.V, self.start.drt
             ,self.peak.TmIdx, self.peak.V, self.peak.drt
                     ,self.peak_position), end='')
        
    #, 'S/P/E:', self.peak.TmIdx, self.end.TmIdx
        

In [172]:
lv1_L =[Trend('init', k_bar=dt[0])]

for k in dt[1:]:
    new, new_trend = lv1_L[-1].update(k)
    print(k[4],':', end='')
    if new == 2:        
        lv1_L.append(Trend('trim', **new_trend))
    elif new == 3:
        for nt in new_trend:
            lv1_L.append(Trend('trim', **nt))
    print('')   
        
len(lv1_L)

c:\users\ariesyun\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:189: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
c:\users\ariesyun\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:200: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


1.0 :
2.0 :
3.0 :
4.0 :
5.0 :
6.0 :
7.0 :
8.0 :
9.0 :
10.0 :
11.0 :
12.0 :
13.0 :
14.0 :
15.0 :
16.0 :
17.0 :
18.0 :
19.0 :
flag=1:drt: 0,status:0,start[0.0,3708.0, 0],peak[0.0,3708.0, 0,ss:1],ES: 0,SS: 0last stick:drt: 1,status:1,rc: 1,S:[  0.0,3708.0, 0], P:[ 17.0,3710.0,-1],pp: 220.0 :
21.0 :
22.0 :
23.0 :
flag=2:drt: 0,status:0,start[0.0,3708.0, 0],peak[0.0,3708.0, 0,ss:1],ES: 0,SS: 0drt: 1,status:0,start[0.0,3708.0, 0],peak[17.0,3710.0,-1,ss:2],ES: 0,SS:1[0.0,3710.0,3708.0, 1]
24.0 :
25.0 :
flag=2:drt: 1,status:0,start[0.0,3708.0, 0],peak[17.0,3710.0,-1,ss:2],ES: 0,SS:1[0.0,3710.0,3708.0, 1]
drt:-1,status:2,rc: 4,S:[ 17.0,3710.0,-1], P:[ 21.0,3700.0, 1],pp: 2zzlm??: {'stick_stack': [<__main__.Stick object at 0x000001BA8BCEC518>], 'ES_s': [], 'SS_s': [<__main__.StdK object at 0x000001BA8BCC2278>], 'status': 0}
26.0 :new trend trim init: start:17.0, ss:1, ES:0, SS:1
new Trend:drt:-1,status:0,start[17.0,3710.0,-1],peak[21.0,3700.0, 1,ss:1],ES: 0,SS:1[17.0,3710.0,3700.0,-1]

27.0 :
28

findIdxInSS: tm= 448.0 verify pp: 9 9
459.0 :
460.0 :
461.0 :
462.0 :
463.0 :
464.0 :
465.0 :
466.0 :
467.0 :
468.0 :
flag=2:drt: 1,status:1,start[377.0,3746.0, 1],peak[468.0,3790.0,-1,ss:10],ES:3[422.0,3776.0,3766.0,-1]
,SS:5[432.0,3786.0,3766.0, 1]
469.0 :
470.0 :
471.0 :
472.0 :
473.0 :
474.0 :
475.0 :
476.0 :
477.0 :
478.0 :
479.0 :
480.0 :
481.0 :
flag=2:drt: 1,status:1,start[377.0,3746.0, 1],peak[468.0,3790.0,-1,ss:11],ES:4[448.0,3786.0,3776.0,-1]
,SS:5[432.0,3786.0,3766.0, 1]
findIdxInSS: tm= 468.0 verify pp: 11 11
482.0 :
483.0 :
484.0 :
flag=1:drt: 1,status:1,start[377.0,3746.0, 1],peak[468.0,3790.0,-1,ss:12],ES:4[448.0,3786.0,3776.0,-1]
,SS:6[456.0,3790.0,3776.0, 1]
last stick:drt:-1,status:0,rc: 5,S:[468.0,3790.0,-1], P:[480.0,3784.0, 1],pp: 5485.0 :
486.0 :
487.0 :
flag=1:drt: 1,status:1,start[377.0,3746.0, 1],peak[468.0,3790.0,-1,ss:12],ES:4[448.0,3786.0,3776.0,-1]
,SS:6[456.0,3790.0,3776.0, 1]
last stick:drt:-1,status:0,rc: 7,S:[468.0,3790.0,-1], P:[480.0,3784.0, 1],pp: 5

868.0 :
869.0 :
870.0 :
871.0 :
flag=1:drt: 1,status:1,start[735.0,3764.0, 1],peak[834.0,3792.0,-1,ss:13],ES:6[834.0,3792.0,3772.0,-1]
,SS:6[816.0,3792.0,3772.0, 1]
last stick:drt: 1,status:1,rc: 1,S:[860.0,3766.0, 1], P:[867.0,3772.0,-1],pp: 4872.0 :
873.0 :
874.0 :
875.0 :
876.0 :
877.0 :
flag=2:drt: 1,status:1,start[735.0,3764.0, 1],peak[834.0,3792.0,-1,ss:13],ES:6[834.0,3792.0,3772.0,-1]
,SS:6[816.0,3792.0,3772.0, 1]
878.0 :
879.0 :
880.0 :
881.0 :
882.0 :
883.0 :
884.0 :
885.0 :
886.0 :
887.0 :
flag=2:drt: 1,status:1,start[735.0,3764.0, 1],peak[834.0,3792.0,-1,ss:14],ES:6[834.0,3792.0,3772.0,-1]
,SS:7[860.0,3772.0,3766.0, 1]
findIdxInSS: tm= 834.0 new trend ss length: 4 15 pp: 11
888.0 :new trend trim init: start:834.0, ss:4, ES:1, SS:2
new Trend:drt:-1,status:1,start[834.0,3792.0,-1],peak[860.0,3766.0, 1,ss:4],ES:1[860.0,3772.0,3766.0, 1]
,SS:2[867.0,3772.0,3768.0,-1]

889.0 :
890.0 :
flag=2:drt:-1,status:1,start[834.0,3792.0,-1],peak[860.0,3766.0, 1,ss:4],ES:1[860.0,3772.0,3766.

1291.0 :
1292.0 :
1293.0 :
1294.0 :
1295.0 :
1296.0 :
1297.0 :
1298.0 :
1299.0 :
1300.0 :


23

In [173]:
tm1,v1 = [], []
for lv1 in lv1_L:
    tm1.append(lv1.start.TmIdx)
    v1.append(lv1.start.V)
    lv1.print_()

drt: 1,status:2,start[0.0,3708.0, 0],peak[17.0,3710.0,-1,ss:3],ES: 0,SS:1[0.0,3710.0,3708.0, 1]
drt:-1,status:2,start[17.0,3710.0,-1],peak[21.0,3700.0, 1,ss:5],ES:1[21.0,3710.0,3700.0, 1]
,SS:2[33.0,3710.0,3702.0,-1]
drt: 1,status:2,start[21.0,3700.0, 1],peak[40.0,3716.0,-1,ss:7],ES:2[40.0,3716.0,3706.0,-1]
,SS:3[49.0,3710.0,3706.0, 1]
drt:-1,status:2,start[40.0,3716.0,-1],peak[94.0,3686.0, 1,ss:9],ES:3[94.0,3700.0,3686.0, 1]
,SS:4[108.0,3700.0,3690.0,-1]
drt: 1,status:2,start[94.0,3686.0, 1],peak[292.0,3752.0,-1,ss:17],ES:6[292.0,3752.0,3738.0,-1]
,SS:8[307.0,3748.0,3738.0, 1]
drt:-1,status:2,start[292.0,3752.0,-1],peak[321.0,3734.0, 1,ss:7],ES:2[321.0,3746.0,3734.0, 1]
,SS:3[329.0,3746.0,3740.0,-1]
drt: 1,status:2,start[321.0,3734.0, 1],peak[347.0,3786.0,-1,ss:7],ES:2[347.0,3786.0,3768.0,-1]
,SS:3[356.0,3776.0,3768.0, 1]
drt:-1,status:2,start[347.0,3786.0,-1],peak[377.0,3746.0, 1,ss:7],ES:2[377.0,3764.0,3746.0, 1]
,SS:3[381.0,3764.0,3750.0,-1]
drt: 1,status:2,start[377.0,3746.0, 1],p

In [171]:
class Trend:
    def __init__(self, method, **kwargs):
        if method == 'init':
            # from P
            self.stick_stack = [Stick('init', k_bar=kwargs['k_bar'])]
            self.start = self.getPoint(kwargs['k_bar'], 0,'init')
            self.end = self.getPoint(kwargs['k_bar'], 0, 'init')
            self.peak = self.getPoint(kwargs['k_bar'], 0, 'init')
            self.peak_position_in_ss = 0
            
            self.ES_stack = []
            self.SS_stack = []
            self.drt = 0            
            self.status = 0 # 0:open; 1:closed

        elif method == 'trim': 
            print('new trend trim init: start:{}, ss:{}, ES:{}, SS:{}'.format(
                kwargs['stick_stack'][0].start.TmIdx
                ,len(kwargs['stick_stack']), len(kwargs['ES_s']), len(kwargs['SS_s'])))
            self.stick_stack = kwargs['stick_stack']
            self.drt = self.stick_stack[0].drt
            self.start = self.getPoint('ss', 0, 'start')
            self.end = self.getPoint('ss', -1, 'end')
        
            # from /
            if len(kwargs['stick_stack']) == 1:
                self.peak = self.getPoint('ss', 0, 'end')
                self.peak_point_in_ss = 1
                self.ES_stack = []
                self.SS_stack = [StdK(**self.stickToStdK(self.stick_stack[0]))]
                    
            #from /\, N
            else:
                self.setPeak()
                self.ES_stack = kwargs['ES_s']
                self.SS_stack = kwargs['SS_s']

            self.status = kwargs['status']
            print('new Trend:', end='')
            self.print_()
            
            if len(self.ES_stack) > 1:
                print('!!!Error!!! ES of new trend > 1.')
            if self.drt == 0:
                print('!!!Error!!! drt = 0.')
     
    def update(self, k_bar):
        self.updateRange(k_bar)
        
        flag = 0 # 0:stain; 1: updated; 2:cloesd;
        new_trend = {}
        
        flag_st, new_stick = self.stick_stack[-1].update(k_bar) 
                    
        if flag_st == 2:
            print('flag=2:', end='')
            self.print_()
        if flag_st == 1:
            print('flag=1:', end='')
            self.print_()
            print('last stick:', end='')
            self.stick_stack[-1].print_()
        
        if self.drt == 0 and flag_st == 2:
            self.stick_stack.append(Stick('trim', **new_stick))
            self.drt = self.stick_stack[0].drt
            self.peak = self.getPoint('ss', 0, 'end')
            self.SS_stack = [StdK(**self.stickToStdK(self.stick_stack[0]))]
            flag = 1
            self.print_()
            
        elif self.status == 0 and flag_st == 2:
            self.stick_stack.append(Stick('trim', **new_stick))
            self.stick_stack[-2].print_()
            if len(self.stick_stack) == 3:
                if (self.stick_stack[1].peak.V - self.stick_stack[0].start.V)*self.drt < 0:
                    self.status = 2
                    new_trend = self.newTrend(0)
                    print('zzlm??:', new_trend)
                    flag = 2
                else:               
                    self.ES_stack = [StdK(**self.stickToStdK(self.stick_stack[1]))]
                    self.status = 1
                    flag = 1
                
            
        elif self.status == 1 and flag_st == 2:            
            # Step1: append to stick_stack
            self.stick_stack.append(Stick('trim', **new_stick))
            tmp_stdk = self.stickToStdK(self.stick_stack[-2])
            
            # 处理eigen_stick:
            if self.drt != tmp_stdk['drt']:
                
                if self.drt == 1:
                    if tmp_stdk['L'] < self.ES_stack[-1].L:
                        if tmp_stdk['H']> self.ES_stack[-1].H:   # case2: 外包
                            if tmp_stdk['L'] < self.start.V:
                                self.status = 2
                                new_trend =  self.newTrend(0)
                                flag = 2
                            else:
                                tmp_stdk['L'] = self.ES_stack[-1].L
                                self.ES_stack.append(StdK(**tmp_stdk))
                                flag = 1
                        elif tmp_stdk['H'] <= self.ES_stack[-1].H:  # case1：反向
                            self.status = 2
                            new_trend = self.newTrend(1)
                            flag = 2
                    elif (tmp_stdk['H'] <= self.ES_stack[-1].H and
                         tmp_stdk['L'] >= self.ES_stack[-1].L):        #  case3: trimUpdate
                        self.ES_stack[-1].L = tmp_stdk['L']
                        self.ES_stack[-1].merged += 1
                        flag = 1
                    elif tmp_stdk['H'] > self.ES_stack[-1].H:     #case4: 进行
                        self.ES_stack.append(StdK(**tmp_stdk))
                        flag = 1
                        
                elif self.drt == -1:
                    if tmp_stdk['H'] > self.ES_stack[-1].H:
                        if tmp_stdk['L'] < self.ES_stack[-1].L:
                            if tmp_stdk['H'] > self.start.V:
                                self.status = 2
                                new_trend =  self.newTrend(0)
                                flag = 2
                            else:
                                tmp_stdk['H'] = self.ES_stack[-1].H
                                self.ES_stack.append(StdK(**tmp_stdk))
                                flag = 1
                        elif tmp_stdk['L'] >= self.ES_stack[-1].L:
                            self.status = 2
                            new_trend = self.newTrend(1)
                            flag = 2
                    elif (tmp_stdk['L'] >= self.ES_stack[-1].L and
                         tmp_stdk['H'] <= self.ES_stack[-1].H):
                        self.ES_stack[-1].H = tmp_stdk['H']
                        self.ES_stack[-1].merged += 1
                        flag = 1
                    elif tmp_stdk['L'] < self.ES_stack[-1].L:
                        self.ES_stack.append(StdK(**tmp_stdk))
                        flag = 1
                        
            
            # 处理syntropy_stick:
            elif self.drt == tmp_stdk['drt']:
                
                if self.drt == 1:
                    if tmp_stdk['H'] > self.SS_stack[-1].H:
                        if self.ES_stack[-1].merged > 1 and tmp_stdk['H'] < self.ES_stack[-1].H:
                            self.status = 2
                            new_trend = self.newTrend(21)
                            flag = 3
                        else:
                            self.SS_stack.append(StdK(**tmp_stdk))
                            flag = 1
                            pp = self.findIdxInSS(self.peak.TmIdx)
                            print('verify pp:', len(self.stick_stack)-1, pp)
                    else:
                        self.SS_stack.append(StdK(**tmp_stdk))
                        flag = 1
                        
                elif self.drt == -1:
                    if tmp_stdk['L'] < self.SS_stack[-1].L:
                        if self.ES_stack[-1].merged > 1  and tmp_stdk['L'] > self.ES_stack[-1].L:
                            self.status = 2
                            new_trend = self.newTrend(21)
                            flag = 3
                        else:
                            self.SS_stack.append(StdK(**tmp_stdk))
                            flag = 1
                            pp = self.findIdxInSS(self.peak.TmIdx)
                            print('verify pp:', len(self.stick_stack)-1, pp)
                    else:
                        self.SS_stack.append(StdK(**tmp_stdk))
                        flag = 1

        return flag, new_trend
               
    def stickToStdK(self, stick_r):
        if stick_r.drt == 1:
            stdk = {'H':stick_r.peak.V, 'L':stick_r.start.V, 'TmIdx':stick_r.start.TmIdx, 'drt': 1}
        elif stick_r.drt == -1:
            stdk = {'L':stick_r.peak.V, 'H':stick_r.start.V, 'TmIdx':stick_r.start.TmIdx, 'drt': -1}
        elif stick_r.drt == 0:
            stdk = {'L':stick_r.start.V, 'H':stick_r.start.V, 'TmIdx':stick_r.start.TmIdx, 'drt': 0}
        return stdk
    
    def getPoint(self, stack, position_in_stack=0, method=None):
        if stack == 'ss':
            if method == 'start':
                drt = self.stick_stack[position_in_stack].drt
                p = Point(self.stick_stack[position_in_stack].start.TmIdx, 
                         self.stick_stack[position_in_stack].start.V,
                         drt)
            elif method == 'end':
                drt = -self.stick_stack[position_in_stack].drt
                p = Point(self.stick_stack[position_in_stack].peak.TmIdx, 
                         self.stick_stack[position_in_stack].peak.V,
                         drt)
        elif stack == 'ES':
            if method == 'start':
                drt = self.ES_stack[position_in_stack].drt
                p = Point(self.ES_stack[position_in_stack].start.TmIdx, 
                         self.ES_stack[position_in_stack].start.V,
                         drt)
            elif method == 'end':
                drt = -self.ES_stack[position_in_stack].drt
                p = Point(self.ES_stack[position_in_stack].peak.TmIdx, 
                         self.ES_stack[position_in_stack].peak.V,
                         drt)
        elif isinstance(stack, str) is False:
            if method == 'H':
                p = Point(stack[4], stack[1], -1)
            elif method =='L':
                p = Point(stack[4], stack[2], 1)
            elif method == 'init':
                p = Point(stack[4], stack[0], 0)
            
        return p
    
    def updateRange(self,k):
        flag = 0
        if self.drt == 1 and k[1] > self.peak.V:
            self.peak = self.getPoint(k, 0, method='H')
            flag = 1
        elif self.drt == -1 and k[2] < self.peak.V:
            self.peak = self.getPoint(k, 0, method = 'L')
            flag = 1
        return flag
    
    def setPeak(self):
        # iterate ss to set peak point and pp in ss
        self.peak = self.getPoint('ss', 0, 'end')
        self.peak_point_in_ss = 1
        pp = 0
        if self.drt == 1 and len(self.stick_stack) >= 3:
            for stick in self.stick_stack[2::2]:
                pp += 2
                if stick.peak.V > self.peak.V:
                    self.peak = self.getPoint('ss', pp, 'end')
                    self.peak_point_in_ss = pp + 1 
        elif self.drt == -1 and len(self.stick_stack) >= 3:
            for stick in self.stick_stack[2::2]:
                pp += 2
                if stick.peak.V < self.peak.V:
                    self.peak = self.getPoint('ss', pp, 'end')
                    self.peak_point_in_ss = pp + 1
        return None
            
    
    def findIdxInSS(self, timeIndex):
        i = 0
        pp = None
        print('findIdxInSS: tm=', timeIndex, end=' ')
        for stick in self.stick_stack:
            if timeIndex == stick.start.TmIdx:
                pp = i
                break
            i += 1
        return pp
    
    def newTrend(self, case):
        if case == 0:
            stick_stack = [self.stick_stack[-2]]
            new_trend = {'stick_stack': stick_stack,
                        'ES_s': [],
                        'SS_s': [StdK(**self.stickToStdK(stick_stack[0]))],
                        'status': 0}
        elif case == 1:
            peak_point_in_stick_stack = self.findIdxInSS(self.ES_stack[-1].TmIdx)
            stick_stack = self.stick_stack[peak_point_in_stick_stack:]
            print('new trend ss length:', len(stick_stack), len(self.stick_stack),'pp:',peak_point_in_stick_stack )
            new_trend = {'stick_stack': stick_stack,
                        'ES_s': self.reduceES(stick_stack),
                        'SS_s': [StdK(**self.stickToStdK(stick)) for stick in stick_stack[::2]],
                        'status': 1}

        elif case == 21:
            pp_TmIdx = 0
            min_v = min(self.stick_stack[-2].start.V, self.stick_stack[-2].peak.V)
            max_v = max(self.stick_stack[-2].start.V, self.stick_stack[-2].peak.V)
            pp_start = self.findIdxInSS(self.ES_stack[-1].TmIdx)
            print('ss idx:', [stick.start.TmIdx for stick in self.stick_stack ])
            print('pp_start:', pp_start)
            for stick in self.stick_stack[-4:pp_start:-2]: 
                stdk = StdK(**self.stickToStdK(stick))
                if self.drt == 1:
                    if stdk.H < max_v and stdk.L <= min_v:
                        min_v = stdk.L
                        pp_TmIdx = stdk.TmIdx
                    elif stdk.H >= max_v:
                        break
                elif self.drt == -1:
                    print('drt/max/min:', self.drt, max_v, min_v)
                    stdk.print_()
                    if stdk.H >= max_v and stdk.L > min_v:
                        max_v = stdk.H
                        pp_TmIdx = stdk.TmIdx
                        print('pp_TmIdx result in for loop:', pp_TmIdx)
                    elif stdk.L <= min_v:
                        break
            print('pp_TmIdx result:', pp_TmIdx)
            stick_stack1 = self.stick_stack[self.findIdxInSS(pp_TmIdx):]
            new_trend1 = {'stick_stack':stick_stack1,
                          'ES_s': self.reduceES(stick_stack1),
                          'SS_s': [StdK(**self.stickToStdK(stick)) for stick in stick_stack1[::2]],
                          'status': 1}
            stick_stack2 = self.stick_stack[self.findIdxInSS(self.ES_stack[-1].TmIdx):self.findIdxInSS(pp_TmIdx)]
            new_trend2 = {'stick_stack':stick_stack2,
                          'ES_s': self.reduceES(stick_stack2, 0),
                          'SS_s': [StdK(**self.stickToStdK(stick)) for stick in stick_stack2[::2]],
                          'status': 2}
            
            new_trend = [new_trend2, new_trend1]
            print('!!!!!!!!!!!!!***********21: new_trend length:', len(new_trend))
        
        return new_trend
    
    def reduceES(self, stick_stack_o, tail=1):
        drt = stick_stack_o[0].drt
        stick_stack = stick_stack_o[:]
        if tail == 1:            
            stick_stack.pop()
        l = len(stick_stack)
        #  if l < 3: stick_stack index (0,1,2) / (0,1) / (0)
        if l == 1:
            ES_stack = []
        else:
            ES_stack = [StdK(**self.stickToStdK(stick_stack[1]))]
        
        if drt == 1 and l >= 3:            
            for stick in stick_stack[3::2]:
                if ES_stack[-1].H < stick.start.V:
                    tmp_stdk = StdK(**self.stickToStdK(stick))
                    if ES_stack[-1].L > stick.peak.V:
                        tmp_stdk.L = ES_stack[-1].L
                    ES_stack.append(tmp_stdk)
                else:
                    ES_stack[-1].trimUpdate(self.stickToStdK(stick))
        elif drt == -1 and l >= 3:
            for stick in stick_stack[3::2]:
                if ES_stack[-1].L > stick.start.V:
                    tmp_stdk = StdK(**self.stickToStdK(stick))
                    if ES_stack[-1].H < stick.peak.V:
                        tmp_stdk.H = ES_stack[-1].H
                    ES_stack.append(tmp_stdk)
                else:
                    ES_stack[-1].trimUpdate(self.stickToStdK(stick))
        return ES_stack
        
    
    def print_(self):
        print('drt:{:>2},status:{},start[{},{:>6},{:>2}],peak[{},{:>6},{:>2},ss:{}]'.format(
        self.drt, self.status, self.start.TmIdx, self.start.V, self.start.drt
        ,self.peak.TmIdx, self.peak.V, self.peak.drt, len(self.stick_stack)),end='')
        if len(self.ES_stack) > 0:
            print(',ES:{}[{},{:>6},{:>6},{:>2}]'.format(
                len(self.ES_stack)
                , self.ES_stack[-1].TmIdx, self.ES_stack[-1].H, self.ES_stack[-1].L, self.ES_stack[-1].drt
                ))
        else:
            print(',ES:', len(self.ES_stack), end='')
        if len(self.SS_stack) > 0:
            print(',SS:{}[{},{:>6},{:>6},{:>2}]'.format(
                len(self.SS_stack)
                , self.SS_stack[-1].TmIdx, self.SS_stack[-1].H, self.SS_stack[-1].L, self.SS_stack[-1].drt
                ))
        else:
            print(',SS:', len(self.SS_stack), end='')
    

In [41]:
ll = [0,29,47,60,63,70]
for l in ll[-4:1:-2]:
    print(l)
    if l ==2:
        break

47


In [81]:
ll = [0,29,47,60,63,70]
ll.pop()
ll

[0, 29, 47, 60, 63]

In [102]:
def tttttt(x):
    tmp = x
    tmp.pop()
    print(x,tmp)

ll = [0,29,47,60,63,70]
tttttt(ll)
print(ll)


[0, 29, 47, 60, 63] [0, 29, 47, 60, 63]
[0, 29, 47, 60, 63]


In [118]:
class Strategy:
    def __init__(self, start_time):
        self.TmS = start_time
        self.stop_tm
        self.stop_price
        self.limit_tm
        self.limit_price
        self.status = 'hold'  # hold/bid/ask
    
    def setStop(self, stop_time):
        self.stop_time = stop_time
        return None
    
    def setLimit(self):
        return None
    
    def xxx(self):
        # 持仓未达标，则开仓；持仓已达标，则标记‘当前为可买入状态’
        return None

In [ ]:
# class Position:

In [158]:
dt_df = pd.read_csv('SHFE.bu1906.txt')
dt_df.columns = ['Date', 'Tm', 'O', 'H', 'L', 'C', 'Vol']
dt_df['TmIdx'] = dt_df.index
dt_df = dt_df.loc[:1300,:]
dt_df.head()

,Date,Tm,O,H,L,C,Vol,TmIdx
0,2018/10/8,09:01:00,3708.0,3752.0,3708.0,3718.0,3128,0
1,2018/10/8,09:02:00,3716.0,3728.0,3712.0,3720.0,652,1
2,2018/10/8,09:03:00,3720.0,3720.0,3708.0,3708.0,768,2
3,2018/10/8,09:04:00,3710.0,3718.0,3710.0,3714.0,548,3
4,2018/10/8,09:05:00,3714.0,3718.0,3710.0,3718.0,678,4


In [159]:
dt = dt_df[['O', 'H', 'L', 'C', 'TmIdx']].values

In [160]:
dt_df['TmIdx'].dtypes

dtype('int64')

In [161]:
# X-axis-data
x_idx = [k[4] for k in dt]
x = dt_df[['TmIdx','Tm']]

In [162]:
# 绘图用插值函数
# input: 
#       1 - TmIdx array 
#       2 - Values array
# output: 按x轴查完值的value array
def interpoint(TmIdx_array, V_array):
    global x
    df_tmp = pd.DataFrame({'V': V_array, 'TmIdx': TmIdx_array})
    df_tmp = df_tmp.merge(x, how='outer', left_on='TmIdx', right_on='TmIdx')
    print(df_tmp['TmIdx'].dtypes)
    df_tmp['TmIdx'].astype('int64')
    df_tmp.sort_values(['TmIdx'], inplace=True)
    return df_tmp['V'].interpolate().values.tolist()

    
#v = [4,5,4]
#tm = [0,100,200]
    
dt_line = interpoint(tm,v)
    

float64


In [174]:
dt_lv1_line = interpoint(tm1, v1)

float64


In [165]:
tm1

[0.0, 93.0, 134.0, 195.0, 260.0, 284.0, 357.0]

In [155]:
kline = Kline('Test1901!')
# 开收低高
oringe_dt = [[k[0], k[3], k[2], k[1]] for k in dt]
kline.add('xx',[k[4] for k in dt], [k[0:4] for k in oringe_dt]
         ,is_datazoom_show=True
         #,datazoom_xaxis_index=[0,1]
         )
#kline.print_echarts_options()
#kline.render() 生成本地html


In [156]:
kline

In [163]:
len(x), len(dt_line)

(1301, 1301)

In [164]:
line = Line('stick')
line.add('xxx', [k[4] for k in dt], dt_line
        ,is_datazoom_show=True
        )

In [175]:
line_lv1 = Line('Trend')
line_lv1.add('xxx', [k[4] for k in dt], dt_lv1_line
        ,is_datazoom_show=True
        )
line_lv1

In [99]:
kline2 = Kline('Test1901-b!')
kline2.add('std', [k[4] for k in dt], std_dt,
          is_datazoom_show=True)

In [176]:
overlap = Overlap()
overlap.add(kline)
overlap.add(line)
overlap.add(line_lv1)
overlap

In [166]:
overlap.render()

In [150]:
grid = Grid()
grid.add(kline, grid_bottom='60%')
grid.add(kline2, grid_top='60%')
grid

In [89]:
import pyecharts
pyecharts.__version__

'0.5.5'

In [26]:
std.TmIdx

0.0

In [20]:
Lv0 = Stick('init', k_bar=dt[0], drt=0)

KeyError: 'H'

In [31]:
Lv0.TmIdx_start

0.0